# install

## I

In [1]:
# Install (versions compatibles + évite le package `keras` standalone)
%pip install -U pip setuptools wheel
%pip install -U tabulate scikit-learn pandas numpy matplotlib seaborn gensim "tensorflow==2.16.1"

# Test TensorFlow (ne fait pas planter le notebook si l'environnement Windows manque de DLL)
try:
	import tensorflow as tf
	print("TensorFlow:", tf.__version__)
	print(tf.reduce_sum(tf.random.normal([1000, 1000])))
except Exception as e:
	print("TensorFlow n'a pas pu se charger (runtime natif / DLL).")
	print("Erreur:", repr(e))
	print("\nPistes courantes (Windows):")
	print(" - Installer/mettre à jour 'Microsoft Visual C++ Redistributable 2015-2022 (x64)'")
	print(" - Éviter Python du Microsoft Store: utiliser python.org/conda + nouveau venv")
	print(" - CPU trop ancien (AVX requis): utiliser un autre environnement (WSL/conda) ou une build no-AVX")

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

^C
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
TensorFlow: 2.16.1
tf.Tensor(1062.485, shape=(), dtype=float32)


## Si echec - diagnostic

In [ ]:
# Diagnostic TensorFlow (sortie courte, utile sous Windows)
import sys, platform, site, pathlib, traceback

print("Python:", sys.version.replace("\n", " "))
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("Machine:", platform.machine())

site_paths = []
try:
    site_paths = site.getsitepackages()
except Exception:
    site_paths = [site.getusersitepackages()]
print("Site-packages:")
for p in site_paths:
    print(" -", p)

# Cherche des traces d'une install cassée (ex: dossier '~ensorflow' -> warning pip)
for p in site_paths:
    base = pathlib.Path(p)
    if base.exists():
        bad = sorted([x.name for x in base.glob("~ensorflow*")])
        if bad:
            print("\nTraces suspectes dans", str(base))
            for name in bad:
                print(" -", name)

print("\nTest import tensorflow...")
try:
    # Évite un vieux module resté en mémoire après uninstall/reinstall
    for k in list(sys.modules.keys()):
        if k == "tensorflow" or k.startswith("tensorflow.") or k.startswith("keras") or k.startswith("tensorboard"):
            sys.modules.pop(k, None)

    import tensorflow as tf
    print("tensorflow module:", tf)
    print(" - __file__:", getattr(tf, "__file__", None))
    spec = getattr(tf, "__spec__", None)
    if spec is not None:
        print(" - spec.origin:", spec.origin)
        print(" - spec.submodule_search_locations:", list(spec.submodule_search_locations or []))
    print("OK TensorFlow:", tf.__version__)
except Exception as e:
    print("ECHEC import TensorFlow:", repr(e))
    print("\nTraceback (résumé):")
    print(traceback.format_exc())

## Si echec - cleanup

In [8]:
# Cleanup TensorFlow/Keras (corrige les résidus type `~ensorflow` + réinitialise l'install)
# À exécuter si l'import TF est cassé / warnings pip `Ignoring invalid distribution ~ensorflow`
import sys, site, pathlib, shutil

print("Python executable:", sys.executable)
if "WindowsApps" in sys.executable:
    print("⚠️ Python Microsoft Store détecté : c'est une source fréquente de soucis avec TF. ")
    print("   Recommandé: Python depuis python.org/Miniconda + venv, puis re-sélectionner le kernel.")

# 1) Désinstall propre via pip (dans le kernel courant)
%pip uninstall -y tensorflow tensorflow-intel tensorflow-cpu keras keras-nightly tensorboard tensorboard-data-server tensorflow-estimator

# 2) Suppression des résidus dans le site-packages UTILISATEUR (pas dans WindowsApps)
user_sp = pathlib.Path(site.getusersitepackages())
print("User site-packages:", user_sp)

patterns = [
    "~ensorflow*", "tensorflow", "tensorflow-*dist-info", "tensorflow_*dist-info",
    "~keras*", "keras", "keras-*dist-info",
    "tensorboard", "tensorboard-*dist-info", "tensorboard_data_server*",
    "tensorflow_estimator*", "tensorflow_io_gcs_filesystem*"
 ]

removed = []
if user_sp.exists():
    for pat in patterns:
        for path in user_sp.glob(pat):
            try:
                if path.is_dir():
                    shutil.rmtree(path, ignore_errors=True)
                else:
                    path.unlink(missing_ok=True)
                removed.append(str(path))
            except Exception as e:
                print("Impossible de supprimer", path, "->", repr(e))
else:
    print("User site-packages introuvable (", user_sp, ")")

print("\nSupprimé (résumé):")
if removed:
    for p in removed[:50]:
        print(" -", p)
    if len(removed) > 50:
        print(f" ... +{len(removed)-50} autres")
else:
    print("(rien à supprimer, ou déjà propre)")

print("\nEnsuite:")
print("1) Redémarre le kernel")
print("2) Relance la cellule Install (TensorFlow==2.16.1)")
print("3) Relance la cellule Diagnostic")

Python executable: c:\Program Files\Python310\python.exe
Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Successfully uninstalled tensorflow-2.16.1
Found existing installation: tensorflow-intel 2.16.1
Uninstalling tensorflow-intel-2.16.1:
  Successfully uninstalled tensorflow-intel-2.16.1
Found existing installation: keras 3.12.1
Uninstalling keras-3.12.1:
  Successfully uninstalled keras-3.12.1
Found existing installation: tensorboard 2.16.2
Uninstalling tensorboard-2.16.2:
  Successfully uninstalled tensorboard-2.16.2
Found existing installation: tensorboard-data-server 0.7.2
Uninstalling tensorboard-data-server-0.7.2:
  Successfully uninstalled tensorboard-data-server-0.7.2
Found existing installation: tensorflow-estimator 2.10.0
Uninstalling tensorflow-estimator-2.10.0:
  Successfully uninstalled tensorflow-estimator-2.10.0
Note: you may need to restart the kernel to use updated packages.
User site-packages: C:\Users\hugog\AppData\Roaming\Python\Pyt

# Importation des librairies

In [1]:
# Suppression des avertissements liés à Scikit-learn
import warnings  # Masquer les avertissements (ex. : FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Librairies générales
import pandas as pd  # Librairie pour la manipulation de données
import numpy as np  # Librairie pour le calcul numérique
import sys  # Fonctions et variables liées à l'interpréteur Python
import copy  # Création de copies d'objets
from numpy import mean, std  # Fonctions de calcul de moyenne et d'écart type
import zipfile  # Traitement de fichiers zip
import os  # Manipulation de fichiers et chemins

# Librairie affichage
import matplotlib.pyplot as plt  # Outils de visualisation 2D
from matplotlib import pyplot  # Interface de la bibliothèque Matplotlib
import seaborn as sns  # Bibliothèque de visualisation de données basée sur Matplotlib

# Scikit-learn pour l'évaluation des modèles
from sklearn.metrics import confusion_matrix  # Matrice de confusion
from sklearn.model_selection import KFold  # Outils de validation croisée
from sklearn.metrics import accuracy_score  # Calcul de l'accuracy
from sklearn.model_selection import train_test_split  # Découpage train/test

# TensorFlow et Keras
import tensorflow as tf  # Librairie de deep learning
import keras  # API haut niveau pour construire et entraîner des modèles de deep learning
from keras import layers  # Modules de couches pour construire des modèles Keras
from keras import models  # Outils pour créer des modèles Keras
from keras import optimizers  # Outils d'optimisation
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Générateur d'images pour l'augmentation des données
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping  # Rappels pour le suivi et l'arrêt précoce
from keras.layers import Input, Dense, Dropout, Flatten, Lambda  # Types de couches Keras
from keras.layers import Conv2D, MaxPooling2D  # Couches convolutionnelles et de pooling
from keras.preprocessing import image  # Outils de prétraitement d'images
from tensorflow.keras.models import Model, load_model  # Définition / chargement de modèles
from keras.datasets import fashion_mnist  # Jeu de données Fashion MNIST
from tensorflow.keras.utils import to_categorical  # Conversion en encodage one-hot
from tensorflow.keras.optimizers import SGD  # Optimiseur Stochastic Gradient Descent
from tensorflow.keras.applications.resnet50 import ResNet50  # Modèle ResNet50 pré-entraîné
from tensorflow.keras.preprocessing import image  # Prétraitement d'images pour les modèles Keras

In [ ]:
# histories: Liste d'history, des objets donnés par model.fit()
#   - history.history: Dictionnaire
#         - 'loss': Loss sur les données d'entraînement par époques, liste de float
#         - 'val_loss': Loss sur des données de validation par époques, liste de float
#         - 'accuracy': Précision sur les données d'entraînement par époques, liste de float
#         - 'val_accuracy': Précision sur des données de validation par époques, liste de float
#   Exemple:
#       histories[0].history['loss'] = [0.8, 0.6, 0.4, ...]

# Cette fonction va servir à afficher la courbe d'accuracy et de loss, pour les données d'entrainement et de validation.
def plot_curves(histories):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Extraction du nombre d'époques d'après l'un des historiques
    # On aurait pu prendre un autre mais cela ne change rien
    epochs = range(len(histories[0].history['loss']))
    
    train_loss = [history.history['loss'] for history in histories]
    val_loss = [history.history['val_loss'] for history in histories]
    train_acc = [history.history['accuracy'] for history in histories]
    val_acc = [history.history['val_accuracy'] for history in histories]

    # Calcul des moyennes et des écart-types pour chaque époque
    mean_loss = np.mean(train_loss, axis=0)
    std_loss = np.std(train_loss, axis=0)

    mean_val_loss = np.mean(val_loss, axis=0)
    std_val_loss = np.std(val_loss, axis=0)
    
    mean_acc = np.mean(train_acc, axis=0)
    std_acc = np.std(train_acc, axis=0)
    
    mean_val_acc = np.mean(val_acc, axis=0)
    std_val_acc = np.std(val_acc, axis=0)

    # Couleurs pour les courbes
    train_color = '#2ca02c'
    val_color = '#9467bd'

    # Courbes de loss avec moyenne et écart-type
    ax1.plot(epochs, mean_loss, color=train_color, label='Train')
    ax1.fill_between(epochs, mean_loss - std_loss, mean_loss + std_loss, color=train_color, alpha=0.2)

    ax1.plot(epochs, mean_val_loss, color=val_color, label='Validation')
    ax1.fill_between(epochs, mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, color=val_color, alpha=0.2)

    # Courbes d'accuracy avec moyenne et écart-type
    ax2.plot(epochs, mean_acc, color=train_color, label='Train')
    ax2.fill_between(epochs, mean_acc - std_acc, mean_acc + std_acc, color=train_color, alpha=0.2)

    ax2.plot(epochs, mean_val_acc, color=val_color, label='Validation')
    ax2.fill_between(epochs, mean_val_acc - std_val_acc, mean_val_acc + std_val_acc, color=val_color, alpha=0.2)
    
    k = len(histories)

    # Titres, labels et légendes
    ax1.set_title(f'Loss (k={k})')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    ax2.set_title(f'Accuracy (k={k})')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.show()